In [1]:
## 获取图片编号
import os

test_id = []

def get_id():
    folder_path = 'datasets/train'
    file_names = os.listdir(folder_path)
    # print(file_names[0])
    for i in range(1, len(file_names)):
        # print(file_names[i].split('.')[0])
        test_id.append(file_names[i].split('.')[0])
get_id()
test_id.sort()
# print(test_id)
print(len(test_id))

1000


In [2]:
## 获取真实准确率
import json

real_data = []

# 示例json文件路径
file_path = "data_vl_train.json"
# 打开并读取JSON文件
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    # print(data)


# 打印读取的数据
for i in range(0, len(data)):
    # print(data[i]['conversations'][0]['value'].split('/')[2].split('.')[0])
    # print(data[i]['conversations'][1]['value'])
    real_data.append( data[i]['conversations'][1]['value'] )
    
print(len(real_data))
# print(real_data)


# 示例json文件路径
file_path = "data_vl_test.json"
# 打开并读取JSON文件
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    # print(data)


# 打印读取的数据
for i in range(0, len(data)):
    # print(data[i]['conversations'][0]['value'].split('/')[2].split('.')[0])
    # print(data[i]['conversations'][1]['value'])
    real_data.append( data[i]['conversations'][1]['value'] )
    
print(len(real_data))

900
1000


In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

# 设定设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型并移动到目标设备
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
).to(device)

# 加载处理器
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

def fun(id):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": "datasets/train/" + id + ".png",
                },
                {"type": "text", "text": "Please analyze the content of the picture and the text in the picture. If you think the combination of the content and the text is harmful (such as violence, pornography, discrimination and other harmful behaviors), please output 1; if you think the combination is harmless, please output 0::"},
            ],
        }
    ]
    
    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    
    # 确保图像和视频输入也被移动到目标设备
    if isinstance(image_inputs, torch.Tensor):
        image_inputs = image_inputs.to(device)
    if isinstance(video_inputs, torch.Tensor):
        video_inputs = video_inputs.to(device)
    
    # 处理文本、图像和视频输入，并确保它们在相同设备上
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    
    # 将所有输入张量移动到目标设备
    for key in inputs:
        if isinstance(inputs[key], torch.Tensor):
            inputs[key] = inputs[key].to(device)
    
    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    
    # 确保生成的ID也在同一设备上
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text
    # print(output_text)


import os

raw_test = []

def start_raw_test():
    for i in range(0, 100):
        # print(file_names[i].split('.')[0])
        raw_test.append(fun(test_id[i]))

start_raw_test()
print(len(raw_test))

print(raw_test)
    

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(


100
[['1'], ['1'], ['0'], ['1'], ['1'], ['1'], ['1'], ['1'], ['0'], ['1'], ['1'], ['1'], ['0'], ['1'], ['1'], ['1'], ['0'], ['0'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['0'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['0'], ['1'], ['0'], ['1'], ['1'], ['1'], ['0'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['0'], ['0']]


In [6]:
# from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

# 设定设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型并移动到目标设备
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
).to(device)

# 加载处理器
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

def fun2(id):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": "datasets/train/" + id + ".png",
                },
                {"type": "text", "text": "Please extract the text in the picture, and judge whether the text is harmful , do not combine the picture. "},
            ],
        }
    ]
    
    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    
    # 确保图像和视频输入也被移动到目标设备
    if isinstance(image_inputs, torch.Tensor):
        image_inputs = image_inputs.to(device)
    if isinstance(video_inputs, torch.Tensor):
        video_inputs = video_inputs.to(device)
    
    # 处理文本、图像和视频输入，并确保它们在相同设备上
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    
    # 将所有输入张量移动到目标设备
    for key in inputs:
        if isinstance(inputs[key], torch.Tensor):
            inputs[key] = inputs[key].to(device)
    
    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    
    # 确保生成的ID也在同一设备上
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text
    # print(output_text)


import os

raw_test_text = []

def start_raw_test_text():
    for i in range(0, 100):
        # print(file_names[i].split('.')[0])
        raw_test_text.append(fun2(test_id[i]))

start_raw_test_text()
print(len(raw_test_text))

print(raw_test_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100
[["when you're feeling horny\nasf but your habibi is on periods let's try a goat"], ['meanwhile at the isis strip club'], ['when your human says\n"who\'s a good girl?"\nand you already know it\'s you'], ['and that was the last\nnativity play my son was\ninvited to take part in.'], ["you can't be racist if there is no other race"], ['when your debit card\ndecline at the abortion\nclinic\n\nand they have to put\nthe baby back in'], [' ladies! if your man is offended by this\nhe is a faggot. end of story'], ["you're a special kinda nuts\naren't you?"], ['father and daughter bonding time is the best\nlike and share if you agree'], ['kermit the frog\ndefinitely not a muslim'], ['how you guys think you looks like\nbut when pados wale pandit jee sees you\ntaba renso'], ['the text in the picture is:\n\nif you think t willl stop\neating palony and viannas\nvoetsek!!!'], ['a smile is worth a\nthousand words'], ['The text in the picture is:\n\n"CAFE\nmuslims offend me'], ['when you forget to\

In [17]:
def acc_rate(real_data, test_data):
    count_0_0 = 0
    count_0_1 = 0
    count_1_0 = 0
    count_1_1 = 0
    for i in range(0, len(real_data)):
        if str(real_data[i]) == '0' and str(test_data[i][0]) == '0':
            count_0_0 += 1
        elif str(real_data[i]) == '0' and str(test_data[i][0]) == '1':
            count_0_1 += 1
        elif str(real_data[i]) == '1' and str(test_data[i][0]) == '0':
            count_1_0 += 1
        else:
            count_1_1 += 1
    print(count_0_0, count_0_1, count_1_0, count_1_1)
    print("raw_acc_rate: ", (count_0_0 + count_1_1)/1000 )
# acc_rate(real_data, raw_test)
# for i in range(0, 100):
#     print(raw_test)

100 533 37 330
raw_acc_rate:  0.43


In [16]:
count_0_0 = 0
count_0_1 = 0
count_1_0 = 0
count_1_1 = 0
for i in range(0, 100):
    print(str(raw_test_text[i][0]), str(raw_test[i][0]))
    
    # if str(raw_test_text[i][0]) == '0' and str(raw_test[i][0]) == '0':
    #     count_0_0 += 1
    # elif str(raw_test_text[i][0]) == '0' and str(raw_test[i][0]) == '1':
    #     count_0_1 += 1
    # elif str(raw_test_text[i][0]) == '1' and str(raw_test[i][0]) == '0':
    #     count_1_0 += 1
    # elif str(raw_test_text[i][0]) == '1' and str(raw_test[i][0]) == '1':
    #     count_1_1 += 1
    # else:
    #     print(str(raw_test_text[i][0]), str(raw_test[i][0]))
print(count_0_0, count_0_1, count_1_0, count_1_1)
# print("raw_acc_rate: ", (count_0_0 + count_1_1)/1000 )

father and daughter bonding time is the best
like and share if you agree 0
trust me... ...they're re all terrorists 1
which one should we butcher first 0
and that was the last time
we let uncle larry babysit 1
9 87 0 0


In [ ]:
 10 

In [3]:
torch.cuda.empty_cache()
